# Adversarial Attack Generator

Based on: https://github.com/Trusted-AI/adversarial-robustness-toolbox/wiki/Get-Started#setup </p>
- Docs: https://adversarial-robustness-toolbox.readthedocs.io/en/latest/index.html


---

## Import Data

In [18]:
# # load data
# import pandas as pd

# # load dataset
# df = pd.read_csv('../data/preprocessed/binary_min_max_combined.csv')
# df.shape
# df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.837186,1.333333e-07,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.840070,1.016667e-06,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.840085,5.416666e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.705516,3.916666e-07,0.000000,0.000003,4.651163e-07,9.153974e-09,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.837156,1.333333e-07,0.000005,0.000000,9.302326e-07,0.000000e+00,0.000242,0.002581,0.00101,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [24]:
# # create smaller dataset
# small_df = df.sample(frac=0.0003, random_state=10)
# small_df.shape

(848, 71)

In [26]:
# # split data
# from sklearn.model_selection import train_test_split

# X = small_df.drop(columns=[' Label'])
# y = small_df[' Label']

# y = pd.get_dummies(y)
# y.columns = ["BENIGN", "MALICIOUS"]
# print(y[:2])

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# print(X_train.shape, X_test.shape)

         BENIGN  MALICIOUS
468003    False       True
1239319    True      False
(636, 70) (212, 70)


---

## Load Data Split

In [1]:
import pandas as pd

X_train = pd.read_csv("../CICIDS2017/train_test_split/X_train_small.csv")
X_test = pd.read_csv("../CICIDS2017/train_test_split/X_test_small.csv")
y_train = pd.read_csv("../CICIDS2017/train_test_split/y_train_small.csv")
y_test = pd.read_csv("../CICIDS2017/train_test_split/y_test_small.csv")

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2000, 70) (500, 70) (2000, 2) (500, 2)


## Import Model

In [5]:
# load model
from tensorflow import keras

model = keras.models.load_model('../CICIDS2017/models/ids_dnn.keras')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 50)             │         3,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 30)             │         1,530 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,238 (63.43 KB)

 Trainable params: 5,412 (21.14 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 10,826 (42.29 KB)

## Convert Model

In [6]:
# convert model to ART -> needed for adversarial attacks
from art.estimators.classification import TensorFlowV2Classifier
import tensorflow as tf

# Define loss function
loss_object = keras.losses.BinaryCrossentropy()
optimizer = keras.optimizers.Adam(learning_rate=0.001)
input_dim = X_train.shape[1] 

@tf.function
def custom_train_step(model, x_batch, y_batch):
    with tf.GradientTape() as tape:
        predictions = model(x_batch, training=True)
        loss = loss_object(y_batch, predictions)
    
    # Compute and apply gradients
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss

# KerasClassifier uses tf.keras.backend.placeholder, which has been removed in TensorFlow 2.10+.so we need to use TensorFlowV2Classifier
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=2,  # Binary classification (0 or 1)
    input_shape=(input_dim,),  # Input shape
    clip_values=(0, 1), # because of the min-max normalization
    optimizer=optimizer, 
    loss_object=loss_object,
    train_step=custom_train_step  # Use default training function
)

# print accuracy
from sklearn.metrics import classification_report, accuracy_score

y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

print(classification_report(y_test, y_pred, target_names=['BENIGN', 'ATTACK']))
print(f"Accuracy : {accuracy_score(y_test, y_pred)*100:.2f}%")

              precision    recall  f1-score   support

      BENIGN       0.99      0.98      0.99       416
      ATTACK       0.92      0.96      0.94        84

   micro avg       0.98      0.98      0.98       500
   macro avg       0.96      0.97      0.96       500
weighted avg       0.98      0.98      0.98       500
 samples avg       0.98      0.98      0.98       500

Accuracy : 98.00%


## Generate Attacks

In [7]:
# convert to numpy -> needed for adversarial attacks
X_test_np = X_test.to_numpy()

### Carlini & Wagner Attack

In [9]:
from art.attacks.evasion import CarliniL2Method

# Create the C&W attack (non-targeted)
attack_cw = CarliniL2Method(classifier=classifier, confidence=0.0, targeted=False)

# Generate adversarial examples on the test set
X_test_adv_cw = attack_cw.generate(x=X_test_np)

# Let's print the shape of the adversarial examples
print(f'Adversarial examples generated. Shape: {X_test_adv_cw.shape}')


C&W L_2:   0%|          | 0/500 [00:00<?, ?it/s]

Adversarial examples generated. Shape: (500, 70)


### FGSM Attack

In [8]:
from art.attacks.evasion import FastGradientMethod

# Create FGSM attack
attack_fgsm = FastGradientMethod(estimator=classifier, eps=0.1)  # ε tune this for stronger/weaker attacks: 0.01 weak, 0.1 balanced, 0.3-0.5 strong, 1 very strong
# the higher the epsilon, the easier it will be detected

# Generate adversarial examples
X_test_adv_fgsm = attack_fgsm.generate(x=X_test_np)

## Evaluate Attacks

In [10]:
# Evaluate the model on adversarial examples
loss, accuracy = model.evaluate(X_test_np, y_test)
print(f'Accuracy on clean examples: {accuracy * 100:.2f}%')

loss_adv, accuracy_adv = model.evaluate(X_test_adv_cw, y_test)
print(f'Accuracy on c&w attack: {accuracy_adv * 100:.2f}%')

loss_adv_fgsm, accuracy_adv_fgsm = model.evaluate(X_test_adv_fgsm, y_test)
print(f'Accuracy on fgsm attack: {accuracy_adv_fgsm * 100:.2f}%')

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9769 - loss: 0.0609  
Accuracy on clean examples: 98.00%
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9475 - loss: 0.0918 
Accuracy on c&w attack: 94.40%
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7376 - loss: 56.8195 
Accuracy on fgsm attack: 72.60%


In [10]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred_adv_cw = model.predict(X_test_adv_cw)
y_pred_adv_cw = (y_pred_adv_cw > 0.5)
y_pred_adv_fgsm = model.predict(X_test_adv_fgsm)
y_pred_adv_fgsm = (y_pred_adv_fgsm > 0.5)

print("Normal Classification Report:")
print(classification_report(y_test, y_pred, target_names=['BENIGN', 'ATTACK']))
print("C&W Classification Report:")
print(classification_report(y_test, y_pred_adv_cw, target_names=['BENIGN', 'ATTACK']))
print("FGSM Classification Report:")
print(classification_report(y_test, y_pred_adv_fgsm, target_names=['BENIGN', 'ATTACK']))

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Normal Classification Report:
              precision    recall  f1-score   support

      BENIGN       0.99      0.98      0.99       416
      ATTACK       0.92      0.96      0.94        84

   micro avg       0.98      0.98      0.98       500
   macro avg       0.96      0.97      0.96       500
weighted avg       0.98      0.98      0.98       500
 samples avg       0.98      0.98      0.98       500

C&W Classification Report:
              precision    recall  f1-score   support

      BENIGN       0.96      0.98      0.97       416
      ATTACK       0.89      0.79      0.84        84

   micro avg       0.95      0.95      0.95       500
   macro avg       0.92      0.88      0.90       500
weighted avg       0.95      0.95      0.95       500
 samples avg       0.95      0.95      0.95       500

FGSM Classification Report:
              precision    recall

## Compare Data

In [11]:
adv_cw_df = pd.DataFrame(X_test_adv_cw)
# set column names
adv_cw_df.columns = X_test.columns
print(adv_cw_df.head(2))
adv_fgsm_df = pd.DataFrame(X_test_adv_fgsm)
# set column names
adv_fgsm_df.columns = X_test.columns
print(adv_fgsm_df.head(2))
norm_df = pd.DataFrame(X_test_np)
print(norm_df.head(2))

    Destination Port   Flow Duration   Total Fwd Packets  \
0           0.585107        0.092050            0.000000   
1           0.002702        0.001042            0.000009   

    Total Backward Packets  Total Length of Fwd Packets  \
0             1.712786e-05                 4.651163e-07   
1             5.066395e-07                 5.066395e-07   

    Total Length of Bwd Packets   Fwd Packet Length Max  \
0                  4.576987e-08            2.417405e-04   
1                  5.066395e-07            5.066395e-07   

    Fwd Packet Length Min   Fwd Packet Length Mean   Fwd Packet Length Std  \
0            2.580645e-03             1.009955e-03            0.000000e+00   
1            5.066395e-07             5.066395e-07            5.066395e-07   

   ...   act_data_pkt_fwd   min_seg_size_forward   Active Mean    Active Std  \
0  ...       0.000000e+00               1.000000  3.417546e-04  0.000000e+00   
1  ...       5.066395e-07               0.999999  5.066395e-07  5.06

## Store Adversarial Instances

In [12]:
# store the adversarial examples
adv_cw_df.to_csv("../CICIDS2017/adversarial_samples/X_test_small_adv_cw.csv", index=False)
adv_fgsm_df.to_csv("../CICIDS2017/adversarial_samples/X_test_small_adv_fgsm.csv", index=False)